# 0.0 - Polynomial Regression

O parâmetro utilizado para treinar o algoritmo **Polynomial Regression** será o grau do polinômio (`degree` - valor padrão `2`).  As características do modelo, bem como os demais parâmetros e atributos podem ser consultados na documentação oficial em:</br></br>https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html

# 1.0 - Importando bibliotecas

In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model as lm
from sklearn import preprocessing as pp

import matplotlib.pyplot as plt

from sklearn.metrics import r2_score, mean_squared_error, root_mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

# 2.0 - Carregando os dados

In [2]:
# Treino
X_train = pd.read_csv('train/X_training.csv')
y_train = pd.read_csv('train/y_training.csv')

# Validação
X_val = pd.read_csv('validation/X_validation.csv')
y_val = pd.read_csv('validation/y_val.csv')

# Teste
X_test = pd.read_csv('test/X_test.csv')
y_test = pd.read_csv('test/y_test.csv')

# 3.0 - Funções

In [3]:
def metrics(y, y_pred, step, algorithm):
    """
    Calcula várias métricas de avaliação de um modelo de regressão e as retorna em um DataFrame.

    Parâmetros:
    -----------
    y : array-like
        Valores da variável resposta.
    
    y_pred : array-like
        Valores preditos pelo modelo.
    
    step : str
        Etapa do processo de modelagem (ex: 'treino', 'validação', 'teste').
    
    algorithm : str
        Nome do algoritmo usado.

    Retorno:
    --------
    df_metrics : pandas.DataFrame
        DataFrame contendo as métricas calculadas:
        - 'Algorithm': Nome do algoritmo.
        - 'Step': Etapa do processo (treino, validação ou teste).
        - 'R2': Coeficiente de determinação.
        - 'MSE': Erro quadrático médio.
        - 'RMSE': Raiz do erro quadrático médio.
        - 'MAE': Erro absoluto médio.
        - 'MAPE': Erro percentual absoluto médio.
    """
    # Calcula as métricas
    r2 = r2_score(y, y_pred)
    mse = mean_squared_error(y, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y, y_pred)
    mape = mean_absolute_percentage_error(y, y_pred)

    # Cria o DataFrame com as métricas
    df_metrics = pd.DataFrame({   
        'Algorithm': [algorithm],
        'Step': [step],
        'R2': [np.round(r2, 4)],
        'MSE': [np.round(mse, 4)],
        'RMSE': [np.round(rmse, 4)],
        'MAE': [np.round(mae, 4)],
        'MAPE': [np.round(mape, 4)]
    })
    
    return df_metrics

def exporta_excel(nome_algoritmo):
    """
    Exporta os DataFrames de métricas para arquivos Excel em diferentes pastas.

    Args:
        nome_algoritmo (str): Nome do algoritmo utilizado.

    Returns:
        None
    """
    etapas = ['train', 'validation', 'test']
    df_list = [df_metrics_train, df_metrics_val, df_metrics_test]

    for etapa, df in zip(etapas, df_list):
        df.to_excel(f'{etapa}/metrics/metrics_{etapa}_{nome_algoritmo}.xlsx', index=False)

# 4.0 - Ajustando os dados

In [4]:
# transformação do formato DataFrame para Series, essencialmente um array unidimensional
y_train, y_val, y_test = [ df.loc[:, 'song_popularity'] for df in [y_train, y_val, y_test] ]

# 5.0 - Desempenho do modelo

## 5.1 - Dados de treino

In [6]:
# Definição do modelo polinomial
poly = pp.PolynomialFeatures()
X_poly_train = poly.fit_transform( X_train )

# Definição do modelo e treinamento
model_train = lm.LinearRegression()
model_train.fit( X_poly_train, y_train )

# Predição sobre os dados de treino
y_pred_train = model_train.predict( X_poly_train )

# Métricas
df_metrics_train = metrics(y_train, y_pred_train, 'Train', 'Polynomial Regression')
df_metrics_train

,Algorithm,Step,R2,MSE,RMSE,MAE,MAPE
0,Polynomial Regression,Train,0.0942,432.9862,20.8083,16.458,8.3505


## 5.2 - Dados de validação

In [7]:
# Definição do modelo polinomial
poly = pp.PolynomialFeatures()
X_poly_train = poly.fit_transform( X_train )
X_poly_val = poly.fit_transform( X_val )

# Definição do modelo e treinamento
model_val = lm.LinearRegression()
model_val.fit( X_poly_train, y_train )

# Predição sobre os dados de validação
y_pred_val = model_val.predict( X_poly_val )

# Métricas
df_metrics_val = metrics(y_val, y_pred_val, 'Validation', 'Polynomial Regression')
df_metrics_val

,Algorithm,Step,R2,MSE,RMSE,MAE,MAPE
0,Polynomial Regression,Validation,0.0665,445.7682,21.1132,16.7499,8.5479


## 5.3 - Ajuste fino dos hiperparâmetros (Etapa de *Fine Tuning*)

In [9]:
# Definir intervalo de parâmetros
degree_array = np.arange(2, 5)

# Criar DataFrame para armazenar resultados
metrics_df = pd.DataFrame(columns=['degree', 'R2', 'MSE', 'RMSE', 'MAE', 'MAPE'])

for degree_value in degree_array:
    # Regressão polinomial
    poly = pp.PolynomialFeatures( degree=degree_value )
    X_poly_train = poly.fit_transform( X_train )
    X_poly_val = poly.fit_transform( X_val )
    
    # Treinamento do algoritmo.
    model_val = lm.LinearRegression()
    model_val.fit( X_poly_train, y_train )
    
    # Predição sobre os dados de validação
    y_pred_val = model_val.predict( X_poly_val )
    
    # Calcular métricas de performance
    r2_tuning  = r2_score( y_val, y_pred_val )
    mse_tuning  = mean_squared_error( y_val, y_pred_val )
    rmse_tuning = root_mean_squared_error( y_val, y_pred_val )
    mae_tuning  = mean_absolute_error( y_val, y_pred_val )
    mape_tuning = mean_absolute_percentage_error( y_val, y_pred_val )
    
    # Armazenar os resultados em uma nova linha como DataFrame
    new_row = pd.DataFrame({
        'degree': [degree_value],
        'R2': [r2_tuning],
        'MSE': [mse_tuning],
        'RMSE': [rmse_tuning],
        'MAE': [mae_tuning],
        'MAPE': [mape_tuning]
    })
    
    # Concatenar a nova linha ao DataFrame existente
    metrics_df = pd.concat([metrics_df, new_row], ignore_index=True)

In [10]:
# Valores das métricas de performance para diferentes graus dos polinômios
metrics_df

,degree,R2,MSE,RMSE,MAE,MAPE
0,2,0.066477,445.768223,21.113224,16.749939,8.547931
1,3,-0.047778,500.326254,22.367974,17.087201,8.678283
2,4,-102.923632,49624.740849,222.766112,36.104220,10.184801


Em particular, estamos interessados em buscar o conjunto de hiper-parâmetros que minimize a **Raiz do Erro Quadrático Médio (RMSE)**. 

In [11]:
# O grau de polinômio que resultou no menor RMSE é 2
best_degree = 2

## 5.4 - Desempenho do modelo para os dados de teste

In [12]:
# Regressão polinomial
poly = pp.PolynomialFeatures( degree=best_degree )
X_poly_train = poly.fit_transform( X_train )
X_poly_val = poly.fit_transform( X_val )
X_poly_test = poly.fit_transform( X_test )

# Treinamento do algoritmo.
model_test = lm.LinearRegression()
model_test.fit( np.concatenate( ( X_poly_train, X_poly_val) ),
                np.concatenate( (y_train, y_val) ) )

# Predição sobre os dados de teste
y_pred_test = model_test.predict( X_poly_test )

# Métricas
df_metrics_test = metrics(y_test, y_pred_test, 'Test', 'Polynomial Regression')
df_metrics_test

,Algorithm,Step,R2,MSE,RMSE,MAE,MAPE
0,Polynomial Regression,Test,0.0909,442.6414,21.039,16.7364,8.277


# 6.0 - Gera as planilhas com os resultados

In [13]:
exporta_excel('polynomial_regression')